Try to train a model and convert it using TF Lite

https://www.tensorflow.org/lite/convert/python_api#converting_a_keras_model_

https://www.tensorflow.org/lite/guide/inference#linux

In [7]:
import numpy as np

import tensorflow as tf

In [7]:
# Create a simple Keras model.
x = [-1, 0, 1, 2, 3, 4]
y = [-3, -1, 1, 3, 5, 7]

model = tf.keras.models.Sequential(
    [tf.keras.layers.Dense(units=1, input_shape=[1])])
model.compile(optimizer='sgd', loss='mean_squared_error')
model.fit(x, y, epochs=20)

In [9]:
# Classic model saving
model.save("classic_model.h5")

# Model saving in TF Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)

744

In [12]:
# Load classic model
model = tf.keras.models.load_model("classic_model.h5")

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="converted_model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the TensorFlow Lite model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
tflite_results = interpreter.get_tensor(output_details[0]['index'])

# Test the classic model on random input data.
tf_results = model(tf.constant(input_data))

# Compare the result.
for tf_result, tflite_result in zip(tf_results, tflite_results):
    print("Testing behavior on both models")
    print(f"Results: {tf_result} vs {tflite_result}")
    np.testing.assert_almost_equal(tf_result, tflite_result, decimal=5)
    print("Success")

Testing behavior on both models
Results: [0.7739396] vs [0.7739396]
Success
